<img src='imgs/kaggle.png' align='left' />

# data = Kaggle IEEE fraud
## n = 10K
## binary classification (is fraud)

### fn -- check for non-nums

In [4]:
# print n unique vals for non-num cols
def col_types(df):
    import numpy as np
    for c in df.columns:
        if not np.issubdtype(df[c].dtype, np.number):
            print(c , ' NOT num...', df[c].nunique())
            if (df[c].nunique() <= 5):
                print(df[c].unique())
            print('')

### csv -> pd

In [3]:
%%time
# ~ 15 secs

import pandas as pd

df = pd.read_csv('train_transaction.csv', low_memory=True)

CPU times: user 15 s, sys: 3.19 s, total: 18.2 s
Wall time: 18.6 s


In [3]:
df

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.50,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.00,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.00,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.00,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.00,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,0,15811047,49.00,W,6550,NaN,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590536,3577536,0,15811049,39.50,W,10444,225.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590537,3577537,0,15811079,30.95,W,12037,595.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590538,3577538,0,15811088,117.00,W,7826,481.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
df.describe()

NameError: name 'df' is not defined

### check types

In [5]:
col_types(df)

ProductCD  NOT num... 5
['W' 'H' 'C' 'S' 'R']

card4  NOT num... 4
['discover' 'mastercard' 'visa' 'american express' nan]

card6  NOT num... 4
['credit' 'debit' nan 'debit or credit' 'charge card']

P_emaildomain  NOT num... 59

R_emaildomain  NOT num... 60

M1  NOT num... 2
['T' nan 'F']

M2  NOT num... 2
['T' nan 'F']

M3  NOT num... 2
['T' nan 'F']

M4  NOT num... 3
['M2' 'M0' nan 'M1']

M5  NOT num... 2
['F' 'T' nan]

M6  NOT num... 2
['T' 'F' nan]

M7  NOT num... 2
[nan 'F' 'T']

M8  NOT num... 2
[nan 'F' 'T']

M9  NOT num... 2
[nan 'F' 'T']



### booleans (T/F) -> integers

In [6]:
for i in range(1,10):
    df['M' + str(i)] = df['M' + str(i)].eq('T').mul(1)

### remove cols w/ large n unique vals

In [7]:
df = df.drop(columns=['P_emaildomain', 'R_emaildomain'], axis=1)

### one hot encode

In [8]:
df = pd.concat([df, pd.get_dummies(df['ProductCD'], drop_first=True)], axis=1)
df = df.drop(columns=['ProductCD'], axis=1)

In [9]:
df = pd.concat([df, pd.get_dummies(df['card4'])], axis=1)
df = df.drop(columns=['card4'], axis=1)

In [10]:
df = pd.concat([df, pd.get_dummies(df['card6'])], axis=1)
df = df.drop(columns=['card6'], axis=1)

### check types again

In [11]:
col_types(df)

### impute

In [12]:
# impute with mean value
for c in df.columns:
    df[c] = df[c].fillna(df[c].mean())

### minify

In [13]:
def reduce_mem_usage(df, verbose=True):
    import numpy as np
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        if verbose:
            print(col)
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: 
        print('calculating memory usage...')
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [14]:
%%time
# 30 seconds
df = reduce_mem_usage(df)

TransactionID
isFraud
TransactionDT
TransactionAmt
card1
card2
card3
card5
addr1
addr2
dist1
dist2
C1
C2
C3
C4
C5
C6
C7
C8
C9
C10
C11
C12
C13
C14
D1
D2
D3
D4
D5
D6
D7
D8
D9
D10
D11
D12
D13
D14
D15
M1
M2
M3
M4
M5
M6
M7
M8
M9
V1
V2
V3
V4
V5
V6
V7
V8
V9
V10
V11
V12
V13
V14
V15
V16
V17
V18
V19
V20
V21
V22
V23
V24
V25
V26
V27
V28
V29
V30
V31
V32
V33
V34
V35
V36
V37
V38
V39
V40
V41
V42
V43
V44
V45
V46
V47
V48
V49
V50
V51
V52
V53
V54
V55
V56
V57
V58
V59
V60
V61
V62
V63
V64
V65
V66
V67
V68
V69
V70
V71
V72
V73
V74
V75
V76
V77
V78
V79
V80
V81
V82
V83
V84
V85
V86
V87
V88
V89
V90
V91
V92
V93
V94
V95
V96
V97
V98
V99
V100
V101
V102
V103
V104
V105
V106
V107
V108
V109
V110
V111
V112
V113
V114
V115
V116
V117
V118
V119
V120
V121
V122
V123
V124
V125
V126
V127
V128
V129
V130
V131
V132
V133
V134
V135
V136
V137
V138
V139
V140
V141
V142
V143
V144
V145
V146
V147
V148
V149
V150
V151
V152
V153
V154
V155
V156
V157
V158
V159
V160
V161
V162
V163
V164
V165
V166
V167
V168
V169
V170
V171
V172
V173
V174
V175
V176
V177

### normalization 

<img src='imgs/std_scale.png' />

In [15]:
# mean normalize
# df = (df-df.mean()) / df.std()

# min-max
# df = (df-df.min())/(df.max()-df.min())

### save

In [17]:
df.to_csvv('fraud_all.csv')

### shuffle, split, save

In [16]:
n_div_4 = int(df.shape[0] / 4)
n_div_4

147635

In [23]:
from sklearn.model_selection import train_test_split

splits = []

df, df_0   = train_test_split(df, test_size=n_div_4, random_state=0, shuffle=True)
splits.append(df_0)

df, df_1   = train_test_split(df, test_size=n_div_4, random_state=0, shuffle=True)
splits.append(df_1)

df_3, df_2 = train_test_split(df, test_size=n_div_4, random_state=0, shuffle=True)
splits.append(df_2)
splits.append(df_3)

In [24]:
# save memory
del df

### save splits

In [25]:
%%time

for i, split in enumerate(splits):
    print(i)
    split.to_csv('fraud_' + str(i) + '.csv')

0
1
2
3


In [6]:
!wc -l *.csv

   25001 fraud_0.csv
   25001 fraud_1.csv
   25001 fraud_2.csv
   25001 fraud_3.csv
  590541 fraud_all.csv
  590541 train_transaction.csv
 1281086 total


In [7]:
!du -h *.csv

 48M	fraud_0.csv
 48M	fraud_1.csv
 48M	fraud_2.csv
 48M	fraud_3.csv
1.1G	fraud_all.csv
655M	train_transaction.csv


In [8]:
!head -2 fraud_0.csv | tail -1

361644,3348644,0,8971373,200.0,6019,583.0,150.0,226.0,143.0,87.0,116.44,238.8,1.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,169.9,28.39,140.1,42.44,69.5,41.6,127.8,0.8335,124.1,146.8,54.62,18.22,59.1,163.5,0,0,0,0,0,0,0,0,0,1.0,1.044,1.076,0.8467,0.877,1.045,1.072,1.027,1.04,0.465,0.4792,0.5576,0.5967,0.9995,0.1226,0.1235,0.1353,0.1365,0.8164,0.8467,0.1306,0.1333,1.035,1.058,0.9775,0.9873,0.000723,0.000803,0.3892,0.4075,0.141,0.1425,0.1313,0.1396,0.5405,0.5776,1.106,1.159,0.1671,0.1785,0.9995,0.1566,0.1688,1.084,1.12,1.022,1.038,0.3845,0.3992,0.1649,0.1715,0.184,0.5757,0.617,1.067,1.119,0.1292,0.1332,0.1356,0.1439,0.8296,0.867,0.1316,0.1423,0.9995,0.9805,0.997,0.0005636,0.3906,0.4087,0.1416,0.146,0.1409,0.1528,0.542,0.5864,1.087,1.143,0.137,0.1454,0.1543,0.8447,0.8823,0.1382,0.1509,1.065,1.1,0.999,0.0009184,0.4036,0.4224,0.1509,0.1552,0.1375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,

In [9]:
!head -2 fraud_1.csv | tail -1

45903,3032903,0,1103448,40.0,16075,514.0,150.0,102.0,330.0,87.0,116.44,15.0,6.0,2.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,4.0,1.0,0.0,7.0,3.0,0.0,169.9,28.39,140.1,42.44,69.5,41.6,519.0,0.75,124.1,146.8,54.62,18.22,257.0,163.5,0,0,0,0,0,0,0,0,0,1.0,1.044,1.076,0.8467,0.877,1.045,1.072,1.027,1.04,0.465,0.4792,0.5576,0.5967,0.9995,0.1226,0.1235,0.1353,0.1365,0.8164,0.8467,0.1306,0.1333,1.035,1.058,0.9775,0.9873,0.000723,0.000803,0.3892,0.4075,0.141,0.1425,0.1313,0.1396,0.5405,0.5776,1.106,1.159,0.1671,0.1785,0.9995,0.1566,0.1688,1.084,1.12,1.022,1.038,0.3845,0.3992,0.1649,0.1715,0.184,0.5757,0.617,1.067,1.119,0.1292,0.1332,0.1356,0.1439,0.8296,0.867,0.1316,0.1423,0.9995,0.9805,0.997,0.0005636,0.3906,0.4087,0.1416,0.146,0.1409,0.1528,0.542,0.5864,1.087,1.143,0.137,0.1454,0.1543,0.8447,0.8823,0.1382,0.1509,1.065,1.1,0.999,0.0009184,0.4036,0.4224,0.1509,0.1552,0.1375,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,40.0,40.0,4